# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [217]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, slides_url, video_url, excerpt, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [218]:
!cat talks.json

[
    {
        "title": "Flexible AutoML: Accelerating AutoML adoption across Amazon",
        "type": "Talk",
        "venue": "2nd International Conference on AI-ML Systems (AIMLSys 2022)", 
        "talk_date": "2022-10-07", 
        "url_slug": "litmus-aimlsys-2022-talk", 
        "teaser_url": "Flexible-AutoML-paradigm.png",
        "video_url": "",
        "slides_url": "https://adivekar-utexas.github.io/files/AIMLSys_2022_demo_vF.pdf",
        "location": "Virtual",
        "excerpt": "Current AutoML systems consider only two scenarios: (i) AutoML model meets performance bar, or (ii) model misses performance bar. Efforts have been dedicated to improving this ratio, but common issues are not addressed: (iii) model missed performance bar by 1% (iv) model is too slow in production (v) Custom model needs 6+ months to deploy. Different AutoML user-personas (Data Scientists, Engineers, Non-Tech) face different issues depending on their background. Flexible AutoML is a paradigm which 

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [219]:
talks = pd.read_json("talks.json")
talks

,title,type,venue,talk_date,url_slug,teaser_url,video_url,slides_url,location,excerpt
0,Flexible AutoML: Accelerating AutoML adoption ...,Talk,2nd International Conference on AI-ML Systems ...,2022-10-07,litmus-aimlsys-2022-talk,Flexible-AutoML-paradigm.png,,https://adivekar-utexas.github.io/files/AIMLSy...,Virtual,Current AutoML systems consider only two scena...
1,"Supervised Learning (Decision Trees, Bagging a...",Lecture,Amazon ML Summer School 2022,2022-06-14,ml-summer-school-2022-lecture,ML-Summer-School-2022-AdaBoost.png,,https://adivekar-utexas.github.io/files/ML-Sum...,Virtual,Amazon ML Summer School 2022 was an initiative...
2,Squeezing the last DRiP: AutoML for Cost-const...,Talk,Amazon Research Days 2021 conference,2021-10-26,drip-ard-2021-talk,DRiP-results.png,,https://adivekar-utexas.github.io/files/Squeez...,Virtual,Current AutoML research aims to minimize the D...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [220]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [221]:
item

title         Squeezing the last DRiP: AutoML for Cost-const...
type                                                       Talk
venue                      Amazon Research Days 2021 conference
talk_date                                            2021-10-26
url_slug                                     drip-ard-2021-talk
teaser_url                                     DRiP-results.png
video_url                                                      
slides_url    https://adivekar-utexas.github.io/files/Squeez...
location                                                Virtual
excerpt       Current AutoML research aims to minimize the D...
Name: 2, dtype: object

In [222]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.talk_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.talk_date) + "-" + item.url_slug 
    year = item.talk_date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    md += f'urlslug: "{item.url_slug }"\n'
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    md += "permalink: /talks/" + html_filename + "\n"
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
    if len(str(item.location)) > 3:
        md += "date: " + str(item.talk_date) + "\n"
    if len(str(item.teaser_url)) > 5:
        md += "teaserurl: '" + item.teaser_url + "'\n"
    if len(str(item.excerpt)) > 5:
        md += "excerpt: '" + html_escape(item.excerpt) + "'\n"
    slides_url = None
    video_url = None
    if len(str(item.slides_url)) > 4:
        slides_url = str(item.slides_url)
        md += "slidesurl: '" + slides_url + "'\n"
    if len(str(item.video_url)) > 4:
        video_url = str(item.video_url)
        md += "videourl: '" + video_url + "'\n"
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"

    more_info = []
    if video_url is not None:
        more_info.append(f'[[Recording]({video_url})]')
    if slides_url is not None:
        more_info.append(f'[[Slides]({slides_url})]')
    if len(more_info) > 0:
        md += f"\n**Resources:** {' '.join(more_info)}\n"
    if len(str(item.excerpt)) > 4:
        md += f"\n**Abstract:**\n{html_escape(item.excerpt)}\n"
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [223]:
!ls ../_talks

2021-10-26-drip-ard-2021-talk.md
2022-06-14-ml-summer-school-2022-lecture.md
2022-10-07-litmus-aimlsys-2022-talk.md


In [224]:
!cat ../_talks/2022-06-14-ml-summer-school-2022-lecture.md

---
title: "Supervised Learning (Decision Trees, Bagging and Boosting algorithms)"
collection: talks
urlslug: "ml-summer-school-2022-lecture"
type: "Lecture"
permalink: /talks/2022-06-14-ml-summer-school-2022-lecture
venue: "Amazon ML Summer School 2022"
date: 2022-06-14
teaserurl: 'ML-Summer-School-2022-AdaBoost.png'
excerpt: 'Amazon ML Summer School 2022 was an initiative which enrolled ~3,000 Indian undergraduate students and helped them learn key ML technologies from Amazon Scientists. I taught a module on Supervised Learning covering Decision Trees, Bagging and Boosting algorithms, followed by a 3-hour Q&amp;A session to clarify student questions.'
slidesurl: 'https://adivekar-utexas.github.io/files/ML-Summer-School-2022-Supervised-Learning-part-2-Abhishek-slides.pdf'
location: "Virtual"
---

**Resources:** [[Slides](https://adivekar-utexas.github.io/files/ML-Summer-School-2022-Supervised-Learning-part-2-Abhishek-slides.pdf)]

**Abstract:**
Amazon ML Summer School 2022 was an init